In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, interact
import h5py
import os
import re

class ImageHandler:
    def __init__(self, dirPath: str):
        self.dirPath = dirPath
        self.images = {}
        self._uploadImages()

    def _uploadImages(self):
        files = os.listdir(self.dirPath)
        for file in files:
            if file.endswith('.h5'):
                match = re.match(r'volume_(\d+)_slice_(\d+)\.h5', file)
                if match:
                    volume_number = match.group(1)
                    if int(volume_number) not in self.images:
                        self.images[int(volume_number)] = []
                    self.images[int(volume_number)].append(os.path.join(self.dirPath, file))

    def displayImage(self, index):
        files = self.images[index]
        files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))  # Extract slice number from filename
        data = []
        for file in files:
            with h5py.File(file, 'r') as h5_file:
                image_data = h5_file['image'][:]  # Adjust the key based on your .h5 structure
                min_value = np.min(image_data)
                max_value = np.max(image_data)
                normalized = (image_data - min_value) / (max_value - min_value)
                data.append(normalized)
        return data
        
    def __len__(self):
        return len(self.images)


# Assuming 'images' is your list of 2D numpy arrays representing brain slices
handler = ImageHandler("/Users/michallejza/Desktop/Data/BRATS/BraTS2020_training_data/content/data")
images = handler.displayImage(20)

def display_image(slice_index):
    plt.imshow(images[slice_index], cmap='gray')
    plt.axis('off')  # Hide the axis
    plt.title(f'Slice: {slice_index}')
    plt.show()

# Create a slider for the number of slices
slice_slider = IntSlider(value=0, min=0, max=len(images)-1, step=1, description='Slice Index')

# Use interact to link the slider with the display function
interact(display_image, slice_index=slice_slider)

FileNotFoundError: [WinError 3] System nie może odnaleźć określonej ścieżki: '/Users/michallejza/Desktop/Data/BRATS/BraTS2020_training_data/content/data'